In [7]:
import torch
from scipy.io.matlab import loadmat
import os
from tqdm import tqdm
from torch import nn

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

labels=loadmat("./imagelabels.mat")["labels"]
model = torch.hub.load("pytorch/vision", "resnet50", weights="DEFAULT")


cuda:0


Using cache found in /home/bagnol/.cache/torch/hub/pytorch_vision_main


In [8]:
model.parameters

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 2

In [ ]:

model.avgpool = nn.AdaptiveAvgPool2d(1)
model.fc = nn.Sequential(
    nn.BatchNorm1d(2048),
    nn.Dropout(p=0.25),
    nn.Linear(in_features=2048, out_features=2048),
    nn.ReLU(),
    nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=2048, out_features=103),
)

PATH = './model_dict.pth'
if os.path.exists(PATH):
    print("loading pretrained model")
    model.load_state_dict(torch.load(PATH))

In [3]:
from PIL import Image
from torchvision import transforms
import numpy as np


transform=transforms.Compose([transforms.Resize([500,500]),transforms.ToTensor()])
dataset_list=[transform(Image.open(os.path.join("./jpg",path))) for path in os.listdir("./jpg/")]

test_loader = torch.utils.data.DataLoader([(dataset_list[i-1],labels[0][i-1]) for i in loadmat("./setid.mat")["trnid"][0]], batch_size=4)
val_loader = torch.utils.data.DataLoader([(dataset_list[i-1],labels[0][i-1]) for i in loadmat("./setid.mat")["valid"][0]], batch_size=4)
train_loader = torch.utils.data.DataLoader([(dataset_list[i-1],labels[0][i-1]) for i in loadmat("./setid.mat")["tstid"][0]], batch_size=96, shuffle=True)

print("train set:\t"+str(len(train_loader))+" total elements with batch size of "+str(train_loader.batch_size))
print("validation set:\t"+str(len(val_loader))+" total elements with batch size of "+str(val_loader.batch_size))
print("test set:\t"+str(len(test_loader))+" total elements with batch size of "+str(test_loader.batch_size))

train set:	65 total elements with batch size of 96
validation set:	255 total elements with batch size of 4
test set:	255 total elements with batch size of 4


In [16]:
import torch.optim as optim
from torch import nn
from torch.optim import lr_scheduler


plist = [
        {'params': model.layer4.parameters(), 'lr': 1e-5},
        {'params': model.fc.parameters(), 'lr': 5e-3}
        ]
optimizer_ft = optim.Adam(plist, lr=0.001)
lr_sch = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(plist, lr=0.001, momentum=0.9)

In [ ]:
model.train()
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    
    tqdm_data=tqdm(enumerate(train_loader, 0),desc="epoch "+str(epoch+1),total=len(train_loader),leave=False)
    for i, data in tqdm_data:
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0], data[1]

        # zero the parameter gradients
        optimizer_ft.zero_grad()

        # forward + backward + optimize
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_ft.step()

        tqdm_data.set_postfix(loss= loss.item(), refresh=False)
        # print statistics
        running_loss += loss.item()

    lr_sch.step()
torch.save(model.state_dict(), PATH)
print('Finished Training')

In [4]:
from matplotlib.pyplot import imshow
import torchvision

model.eval()
it=iter(val_loader)
pred=list()
true_labels=list()
class_accuracy={str(i+1):{"true":0, "total":0} for i in range(102)}
accuracy=0
for i, data in tqdm(enumerate(val_loader, 0),total=len(val_loader)):
    val_images, val_labels = data[0], data[1]

    #imshow(torchvision.utils.make_grid(val_images).permute(1,2,0))
    #print('GroundTruth: ', ' '.join(f'{str(val_labels[j].item()):5s}' for j in range(4)))
    outputs = model(val_images)
    _, predicted = torch.max(outputs, 1)
    #print('Predicted: ', ' '.join(f'{str(predicted[j].item()):5s}' for j in range(4)))
    for b in range(4):
        if val_labels[b].item()==predicted[b].item():
            accuracy+=1
            class_accuracy[str(val_labels[b].item())]["true"]+=1
        class_accuracy[str(val_labels[b].item())]["total"]+=1
        true_labels.append(val_labels[b].item())
        pred.append(predicted[b].item())
accuracy=accuracy/(4*len(val_loader))

100%|██████████| 255/255 [03:03<00:00,  1.39it/s]


In [5]:
print("Accuracy:\t"+str(accuracy))
print(f"\nClasses accuracy:\t{class_accuracy}")

Accuracy:	0.013725490196078431

Classes accuracy:	{'1': {'true': 0, 'total': 10}, '2': {'true': 0, 'total': 10}, '3': {'true': 0, 'total': 10}, '4': {'true': 0, 'total': 10}, '5': {'true': 0, 'total': 10}, '6': {'true': 0, 'total': 10}, '7': {'true': 0, 'total': 10}, '8': {'true': 0, 'total': 10}, '9': {'true': 0, 'total': 10}, '10': {'true': 0, 'total': 10}, '11': {'true': 0, 'total': 10}, '12': {'true': 0, 'total': 10}, '13': {'true': 0, 'total': 10}, '14': {'true': 0, 'total': 10}, '15': {'true': 0, 'total': 10}, '16': {'true': 0, 'total': 10}, '17': {'true': 0, 'total': 10}, '18': {'true': 0, 'total': 10}, '19': {'true': 0, 'total': 10}, '20': {'true': 0, 'total': 10}, '21': {'true': 0, 'total': 10}, '22': {'true': 0, 'total': 10}, '23': {'true': 0, 'total': 10}, '24': {'true': 0, 'total': 10}, '25': {'true': 0, 'total': 10}, '26': {'true': 0, 'total': 10}, '27': {'true': 0, 'total': 10}, '28': {'true': 0, 'total': 10}, '29': {'true': 0, 'total': 10}, '30': {'true': 0, 'total': 10}